In [11]:
import import_ipynb
from tweet_prepossesssing_and_clustering import is_english,extract_hashtag,remove_stop_words,lemmatize,tokenize,generate_bow,count_dict,tf_idf_vector,one_hot_encoding_vector,cosine_similarity,jaccard_similarity,dice_similarity,k_means_clustering,most_frequent_words,generate_word_cloud,iterative_clustering
from chainning_and_analysis import calculate_silhouette_scores,calculate_cluster_weights,event_clusters_filter,generate_clusters,write_clusters_to_text_and_hashtags,read_clusters_from_file,read_hashtags_from_file,centroid,calculate_similarity,textual_similarity,create_bipartite_graph,apply_hungarian_method,create_cluster_chains,get_folder_names,write_cluster_chains,analyze_sentiment,plot_pie_chart,generate_emoji_image,generate_word_cloud,analyze_and_visualize_clusters

In [12]:
from sklearn.metrics import silhouette_score
import numpy as np
import emoji
from math import sqrt
import pandas as pd
import re
import networkx as nx
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import pos_tag
import string
from langdetect import detect
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from collections import Counter
import operator
from textblob import TextBlob
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch
from sklearn.metrics import pairwise_distances
from collections import defaultdict
from scipy.optimize import linear_sum_assignment
import emojis
from PIL import Image, ImageDraw, ImageFont

In [13]:
import nltk
nltk.download('punkt')  # For tokenization
nltk.download('wordnet')  # For lemmatization
nltk.download('averaged_perceptron_tagger')  # For POS tagging
nltk.download('omw-1.4')  # Optional, for WordNet lemmatizer's additional language support

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prabha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Prabha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Prabha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Prabha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
import nltk
nltk.download('averaged_perceptron_tagger_eng', download_dir='C:\\nltk_data')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [15]:
def process_text_files(input_folder):
    # Ensure the folder exists
    if not os.path.exists(input_folder):
        print(f"Input folder '{input_folder}' does not exist.")
        return

    # Iterate through all text files in the folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            print(f"Processing file: {filename}")
            file_path = os.path.join(input_folder, filename)
            
            # Read the file into a DataFrame
            df = pd.read_csv(file_path, names=['Text'])
                   
            # Handle missing or non-string values
            df['Text'] = df['Text'].fillna('').astype(str) # Replace NaNs with empty strings and ensure type is string
            
            # Filter out non-English tweets
            df = df[df['Text'].apply(is_english)]
            
            # Extract hashtags and tokenize the tweets
            extract_hashtag(df)
            tokenize(df)
            
            # Remove stop words and filter out empty tokens
            df['filtered_tokens'] = df['Tokenized_Tweets'].apply(remove_stop_words)
            df = df[df['filtered_tokens'].apply(lambda x: len(x) > 0)]
            df.reset_index(drop=True, inplace=True)
            
            # Generate Bag of Words and TF-IDF vectors
            bag_of_words = generate_bow(df['filtered_tokens'])
            vector = tf_idf_vector(df['filtered_tokens'].tolist(), bag_of_words.keys())
            vector = np.array(vector)
            
            # # Perform clustering and evaluate clusters
            cluster, no_of_clusters = iterative_clustering(vector, 0.1)
            cluster_qualities = calculate_silhouette_scores(np.array(vector), np.array(cluster))
            cluster_weights = calculate_cluster_weights(vector, cluster)
            event_clusters = event_clusters_filter(cluster_qualities, cluster_weights, 0.7, 6.0)
            
            # # Generate clusters of tweets and hashtags
            event_clusters_tweets, event_clusters_hashtags = generate_clusters(
            list(df['filtered_tokens']), cluster, event_clusters, list(df['Hashtag'])
             )
            no_of_clusters = len(event_clusters)
            
            if no_of_clusters < 1:
                print(f"No valid clusters found in file: {filename}")
                continue
            
            # # Extract the most frequent words in each cluster
            word_sets = most_frequent_words(event_clusters_tweets, no_of_clusters)
            
            # # Create output directory for the processed clusters
            folder_name = os.path.splitext(filename)[0]
            output_folder = os.path.join('event_tweets', folder_name)
            os.makedirs(output_folder, exist_ok=True)
            
            # # Write event clusters and hashtags to files
            event_cluster_file = os.path.join(output_folder, 'event_clusters.txt')
            event_hashtags_file = os.path.join(output_folder, 'event_hashtags.txt')
            write_clusters_to_text_and_hashtags(event_clusters_tweets, event_clusters_hashtags, event_cluster_file, event_hashtags_file)
            
            # # Generate word clouds for each cluster
            for i in range(no_of_clusters):
                 print(f"Top words for cluster {i + 1}:")
                 print(sorted(word_sets[i].items(), key=operator.itemgetter(1), reverse=True)[:10])
                 generate_word_cloud(word_sets[i], i, os.path.join(output_folder, f"img{i}"))


In [16]:
process_text_files('output_tweets')

Processing file: batch_1.txt
Top words for cluster 1:
[('trinbago', 7), ('knight', 7), ('riders', 7), ('time', 4), ('titles', 4), ('tkr', 2), ('champions', 2), ('times', 1), ('tag', 1), ('answer', 1)]
Processing file: batch_10.txt
No valid clusters found in file: batch_10.txt
Processing file: batch_11.txt
No valid clusters found in file: batch_11.txt
Processing file: batch_12.txt
No valid clusters found in file: batch_12.txt
Processing file: batch_13.txt
No valid clusters found in file: batch_13.txt
Processing file: batch_14.txt
Top words for cluster 1:
[('b', 2)]
Processing file: batch_15.txt
Top words for cluster 1:
[('b', 8), ('join', 4), ('answer', 2), ('ans', 2), ('k', 1), ('l', 1), ('rahul', 1)]
Top words for cluster 2:
[('fuck', 2), ('fuckkkkkkk', 1)]
Top words for cluster 3:
[('replacing', 2)]
Processing file: batch_16.txt
Top words for cluster 1:
[('ans', 4), ('join', 4), ('b', 3), ('answer', 1)]
Processing file: batch_17.txt
Top words for cluster 1:
[('v', 9), ('kent', 3), ('

In [17]:
folder_path = os.path.join(os.getcwd(),'event_tweets')
timestamps = get_folder_names(folder_path)

In [18]:
resulting_chains = create_cluster_chains(timestamps,folder_path,0.5)

batch_1 and batch_14
batch_14 and batch_15
batch_15 and batch_16
batch_16 and batch_17
batch_17 and batch_18
batch_18 and batch_19
batch_19 and batch_20
batch_20 and batch_3
batch_3 and batch_5
batch_5 and batch_6
batch_6 and batch_7
batch_7 and batch_8


In [19]:
def create_clusters_dict(folder_path):
    clusters_dict = {}
    folders = sorted(os.listdir(folder_path))
    for i in range(len(folders)-1):
        current_folder = folders[i]
        next_folder = folders[i+1]

        current_file_path = os.path.join(folder_path, current_folder, 'event_clusters.txt')
        next_file_path = os.path.join(folder_path, next_folder, 'event_clusters.txt')

        current_clusters = read_clusters_from_file(current_file_path)
        next_clusters = read_clusters_from_file(next_file_path)

        clusters_dict[current_folder] = current_clusters
        clusters_dict[next_folder] = next_clusters

    return clusters_dict

In [20]:
clusters_dict = create_clusters_dict(os.path.join(os.getcwd(),'event_tweets'))

In [21]:
write_cluster_chains(resulting_chains,os.getcwd(),clusters_dict)

In [22]:
cluster_chains_folder = os.path.join(os.getcwd(),'cluster_chains')
output_folder = os.path.join(os.getcwd(),'Sentiment_analysis')
analyze_and_visualize_clusters(cluster_chains_folder, output_folder)